In [ ]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class FireDetectionNet:
	@staticmethod

	def build(width, height, depth, classes):
	
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		model.add(SeparableConv2D(16, (7, 7), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		model.add(SeparableConv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
  
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(SeparableConv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		model.add(Flatten())
		model.add(Dense(128))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.3))
	
		model.add(Dense(classes))
		model.add(Activation("softmax"))
	
		return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
FIRE_PATH = '/content/drive/My Drive/fire_detection/Fire Dataset'
NON_FIRE_PATH = '/content/drive/My Drive/fire_detection/Non-Fire Dataset'

CLASSES = ["Fire", "Non-Fire"]

TRAIN_SPLIT = 0.75
TEST_SPLIT = 0.25

INIT_LR = 1e-3
BATCH_SIZE = 32
NUM_EPOCHS = 120

SAMPLE_SIZE = 50

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import sys

def load_dataset(dataset_path):

  image_paths = list(paths.list_images(dataset_path))
  image_data = []

  for image_path in image_paths:
    
    try:
      cur_image = cv2.imread(image_path)
      cur_image = cv2.resize(cur_image, (128, 128))
      cur_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except:
      break

    image_data.append(cur_image)

  return np.array(image_data, dtype="float32")

print("[INFO] loading fire data...")
fire_data = load_dataset(FIRE_PATH)
print("[INFO] loading non-fire data...")
non_fire_data = load_dataset(NON_FIRE_PATH)
print("[INFO] Complete!")

In [ ]:
fire_labels = np.ones((fire_data.shape[0],))
non_fire_labels = np.zeros((non_fire_data.shape[0],))

In [ ]:
image_data = np.vstack([fire_data, non_fire_data])
labels = np.hstack([fire_labels, non_fire_labels])

In [ ]:
image_data /= 255
labels = to_categorical(labels, num_classes=2)
class_totals = labels.sum(axis=0)
classWeight = classTotals.max() / classTotals
classWeight = {i : classWeight[i] for i in range(len(classWeight))}

In [ ]:
print(classWeight)
print(type(classWeight))
print(len(classWeight))

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size= TEST_SPLIT, random_state=42)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
%matplotlib inline
plt.figure()
plt.imshow(trainX[1])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
from tensorflow import keras
aug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

print("Compiling model...")
opt = keras.optimizers.Adam(learning_rate=INIT_LR)
model = FireDetectionNet.build(width=128, height=128, depth=3, classes=2)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

print("Training network...")

H = model.fit(aug.flow(trainX, trainY, batch_size= BATCH_SIZE),	
    					validation_data=(testX, testY), 
							steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
							epochs= NUM_EPOCHS, 
							class_weight=classWeight,
							verbose=1, 
							shuffle=True)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

N = np.arange(0,  NUM_EPOCHS)
plt.style.use("ggplot")
fig = plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

In [ ]:
print(predictions.shape)
print(predictions[:1])

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(testX[i], cmap=plt.cm.binary)
    if testY[i][0] == 1:
      if predictions[i][0] > 0.5:
        plt.xlabel("Non-Fire", color='green')
      else:
        plt.xlabel("Non-Fire", color='red')
    else:
      if predictions[i][1] > 0.5:
        plt.xlabel("Fire", color='green')
      else:
        plt.xlabel("Fire", color='red')

plt.show()

In [ ]:
MODEL_PATH = '/content/drive/My Drive/fire_detection/fire_data'
model.save(MODEL_PATH)

In [ ]:
model.summary()

In [ ]:
model.save("/content/drive/My Drive/fire_detection/fire_model.h5")
print("Saved model to drive")